In [33]:
import gensim
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield line.split()
                
                
path = '/Users/MK/Desktop/Minchul/Machine Learning/LDA/temporary_data/'
#file = 'CK12original_text.txt'
sentences = MySentences(path)

dictionary = corpora.Dictionary(sentences)

In [34]:
print 'prior to removing words that only appear once:'
print dictionary
#remove infrequent words in the dictionary
#remove stopwords in the dictionary
stoplist = set('for a of the and to in'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.compactify() # remove gaps in id sequence after words that were removed
dictionary
print 'after removing: '
print dictionary

prior to removing:
Dictionary(31526 unique tokens: [u'woods', u'spiders', u'hanging', u'httpswwwgenomegov', u'woody']...)
after removing: 
Dictionary(17629 unique tokens: [u'xylem', u'flickrbalancedcrafts', u'writings', u'systematic', u'halebopp']...)


In [35]:
print(dictionary.token2id)

{u'xylem': 0, u'flickrbalancedcrafts': 1, u'writings': 2, u'systematic': 3, u'halebopp': 4, u'yellow': 5, u'httpwwwdnalcorgviewanimationgenesdontblendhtml': 6, u'four': 7, u'marfan': 8, u'prefix': 9, u'chameleons': 10, u'spiders': 12, u'hanging': 72, u'woody': 14, u'centimeter': 16, u'localized': 17, u'phenotypic': 17262, u'looking': 5705, u'suborganelle': 19, u'electricity': 20, u'rupture': 11523, u'similarity': 8583, u'tephra': 23, u'unanswered': 24, u'fingernails': 25, u'lori': 26, u'demoted': 27, u'hermann': 28, u'inwards': 29, u'immature': 30, u'meadows': 31, u'cytochrome': 32, u'sinking': 33, u'digit': 34, u'hormone': 35, u'shielding': 36, u'osteichthyes': 37, u'propane': 38, u'chelicerates': 8586, u'regional': 40, u'pierce': 11525, u'pigment': 43, u'foul': 45, u'experimentally': 46, u'bringing': 309, u'cjun': 48, u'hare': 15230, u'wooded': 50, u'basics': 51, u'internally': 52, u'prize': 53, u'vibrational': 54, u'andre': 8589, u'wooden': 56, u'haber': 57, u'persisted': 58, u'wood

In [39]:
#if you do not care about the memory, just do
# corpus = [dictionary.doc2bow(word) for word in sentences]

class MyCorpus(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield dictionary.doc2bow(line.lower().split())

path = '/Users/MK/Desktop/Minchul/Machine Learning/LDA/temporary_data/'
corpus = MyCorpus(path)                

In [40]:
corpora.MmCorpus.serialize('/Users/MK/Desktop/Minchul/Machine Learning/LDA/testcorpus.mm', corpus)
#load_corpus = corpora.MmCorpus('/Users/MK/Desktop/Minchul/Machine Learning/LDA/testcorpus.mm')


In [41]:
print corpus

In [71]:
lda = gensim.models.LdaModel(corpus, id2word = dictionary, num_topics=100)
#lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every=1, chunksize=10000, passes=1)

In [67]:
lda.print_topics(100)

[(0,
  u'0.244*force + 0.057*amount + 0.055*gravity + 0.042*nuclear + 0.042*due + 0.040*formula + 0.035*series + 0.031*opposite + 0.026*area + 0.024*iron'),
 (1,
  u'0.179*g + 0.048*floor + 0.040*bases + 0.039*enzyme + 0.038*suppose + 0.037*mechanism + 0.034*loss + 0.033*skin + 0.030*hold + 0.029*square'),
 (2,
  u'0.151*substance + 0.106*metal + 0.072*substances + 0.069*toward + 0.060*note + 0.041*equivalent + 0.034*mol + 0.030*sides + 0.030*held + 0.025*binding'),
 (3,
  u'0.501*figure + 0.294*shown + 0.051*divided + 0.019*spend + 0.012*connecting + 0.010*whenever + 0.009*accuracy + 0.007*ophiuroidea + 0.007*hclg + 0.007*smonoclinic'),
 (4,
  u'0.237*motion + 0.082*space + 0.064*ionic + 0.058*place + 0.054*effect + 0.037*fall + 0.036*processes + 0.031*allow + 0.026*absorbed + 0.022*projectile'),
 (5,
  u'0.183*field + 0.131*magnetic + 0.101*discoveries + 0.090*using + 0.035*battery + 0.032*quantum + 0.027*applied + 0.026*basic + 0.022*mixture + 0.022*good'),
 (6,
  u'0.092*average + 

In [70]:
#'courtesy is mapped to 3712 in the dictionary, 
# flickrbalancedcrafts is mapped to 1 in the dictionary. 
# if we have a document with 100 words of courtesy, and 1 word of flickrbalancedcrafts,
# the document is represented as below, and is close to the topic 18, which contains the word coutesy with weight 0.085 
doc_bow = [(3712, 100), (1, 1)]
lda[doc_bow]

[(18, 0.9804901960785749)]